# Predicting house price using linear regression

In [1]:
# Loading the libraries
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [2]:
# Download the dataset: 
(x_train, y_train), (x_train, x_test) = boston_housing.load_data()

57344/57026 [==============================] - 1s 14us/step
